In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-r49xplii
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-r49xplii
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4305 sha256=d3245cc16e68155e26801e64da4ae730fc325fd6bf4b71e6e0acb4efe541f35c
  Stored in directory: /tmp/pip-ephem-wheel-cache-l84d_ckk/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin


created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%writefile vectoradd.cu
#include <iostream>
#include <cuda_runtime.h>
using namespace std;
__global__ void addVectors(int* A, int* B, int* C, int n) {
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  if (i < n) {
  C[i] = A[i] + B[i];
}
}


void takeinput(int *A,int *B,int n)
{
  cout<<"Enter "<<n<<" elements for vector A :";
  for (int i=0;i<n;i++)
  {
    cin>>A[i];
  }
  cout<<"\nEnter "<<n<<" elements for vector B :";
  for (int i=0;i<n;i++)
  {
    cin>>B[i];
  }
}


int main() {
  int n = 10;
  int* A, * B, * C;
  int size = n * sizeof(int);
  // Allocate memory on the host
  cudaMallocHost(&A, size);
  cudaMallocHost(&B, size);
  cudaMallocHost(&C, size);
  // Initialize the vectors
  
  takeinput(A,B,n);
  
  // Allocate memory on the device
  int* dev_A, * dev_B, * dev_C;
  cudaMalloc(&dev_A, size);
  cudaMalloc(&dev_B, size);
  cudaMalloc(&dev_C, size);
  // Copy data from host to device
  cudaMemcpy(dev_A, A, size, cudaMemcpyHostToDevice);
  cudaMemcpy(dev_B, B, size, cudaMemcpyHostToDevice);
  // Launch the kernel
  //int blockSize = 256;
  //int numBlocks = (n + blockSize - 1) / blockSize;

  addVectors <<<1,n>>>(dev_A,dev_B,dev_C,n);

  // Copy data from device to host
  cudaMemcpy(C, dev_C, size, cudaMemcpyDeviceToHost);
  // Print the results
  cout<<"\nVector Addition Result  :";
  for (int i = 0; i < n; i++) {
    cout << C[i] << " ";
  }
  cout << endl;
  // Free memory
  cudaFree(dev_A);
  cudaFree(dev_B);
  cudaFree(dev_C);
  cudaFreeHost(A);
  cudaFreeHost(B);
  cudaFreeHost(C);
  return 0;
}


Overwriting vectoradd.cu


In [ ]:
!nvcc vectoradd.cu -o vectoradd

In [ ]:
!./vectoradd

Enter 10 elements for vector A :1 2 3 4 5 6 7 8 9 10

Enter 10 elements for vector B : 80 50 40 5 0 70 9 0 70 3

Vector Addition Result  :81 52 43 9 5 76 16 8 79 13 


In [ ]:
%%writefile "matrixmult.cu"
#include <cuda_runtime.h>
#include <iostream>
using namespace std;

__global__ void matmul(int* A, int* B, int* C, int N) {
int Row = blockIdx.y*blockDim.y+threadIdx.y;
int Col = blockIdx.x*blockDim.x+threadIdx.x;
if (Row < N && Col < N) {
  int Pvalue = 0;
  for (int k = 0; k < N; k++) {
    Pvalue += A[Row*N+k] * B[k*N+Col];
  }
  C[Row*N+Col] = Pvalue;
  }
}

void take_input(int *A,int *B,int N)
{
  cout<<"Enter "<<N*N<<" elements in matrix A :";

  for (int i=0;i<N*N;i++)
  {
    cin>>A[i];
  }

  cout<<"Enter "<<N*N<<" elements in matrix B :";

  for (int i=0;i<N*N;i++)
  {
    cin>>B[i];
  }

}

int main() 
{
  int N = 2;
  int size = N * N * sizeof(int);
  int* A,* B,* C;
  int* dev_A, * dev_B, * dev_C;
  
  cudaMallocHost(&A, size);
  cudaMallocHost(&B, size);
  cudaMallocHost(&C, size);
  cudaMalloc(&dev_A, size);
  cudaMalloc(&dev_B, size);
  cudaMalloc(&dev_C, size);
 
  // Initialize matrices A and B

  take_input(A,B,N);

  cudaMemcpy(dev_A, A, size, cudaMemcpyHostToDevice);
  cudaMemcpy(dev_B, B, size, cudaMemcpyHostToDevice);
  
  dim3 dimBlock(2,2);
  dim3 dimGrid(N/dimBlock.x, N/dimBlock.y);

  matmul<<<dimGrid,dimBlock>>>(dev_A, dev_B, dev_C, N);
  cudaMemcpy(C, dev_C,size,cudaMemcpyDeviceToHost);

  // Print the result

  cout<<"Result of matrix multiplication :\n";

  for (int i = 0; i < N; i++)
  {
    for (int j = 0; j < N; j++)
    {
      cout << C[i*N+j] << " ";
    }
    cout << "\n";
  }
  // Free memory
  cudaFree(dev_A);
  cudaFree(dev_B);
  cudaFree(dev_C);
  cudaFreeHost(A);
  cudaFreeHost(B);
  cudaFreeHost(C);
  return 0;
}


Writing matrixmult.cu


In [ ]:
!nvcc matrixmult.cu -o matrixmult

In [ ]:
!./matrixmult

Enter 4 elements in matrix A :5 6 7 8
Enter 4 elements in matrix B : 1 2 3 4
Result of matrix multiplication :
23 34 
31 46 
